**Installation**

In [1]:
!pip install datasets
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 30.7 MB/s 
     |████████████████████████████████| 140 kB 66.5 MB/s 
     |████████████████████████████████| 1.1 MB 50.5 MB/s 
     |████████████████████████████████| 101 kB 9.7 MB/s 
     |████████████████████████████████| 212 kB 67.5 MB/s 
     |████████████████████████████████| 596 kB 66.5 MB/s 
     |████████████████████████████████| 127 kB 71.6 MB/s 
     |████████████████████████████████| 144 kB 73.6 MB/s 
     |████████████████████████████████| 271 kB 60.0 MB/s 
     |████████████████████████████████| 94 kB 3.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
ERROR: pip's dep

**Import packages**

In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer
import torch
import numpy as np
import collections
from tqdm.auto import tqdm

**Loading the dataset**

In [3]:
data = load_dataset('Shayanvsf/ParSQuAD')


Please Enter your desire mode(manual / automatic) :automatic


Extracting data files:   0%|          | 0/4 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

/root/.cache/huggingface/datasets/downloads/e21320fb1d770887774bdc0a339f8a74ef8a2e52551819f9c37920181a665958


Generating validation split: 0 examples [00:00, ? examples/s]

/root/.cache/huggingface/datasets/downloads/7d8284da0afe9cd671916972e3002ffbd8845b6e2f97f0e0b8684ae47b628e18
Dataset par_s_qu_ad downloaded and prepared to /root/.cache/huggingface/datasets/Shayanvsf___par_s_qu_ad/ParSQuAD/1.0.0/124ac9a0ce7760ed368457a321c510f7c5592f9fa8026e6643cfe3d551b5d0d3. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
data['train'][0]

{'answers': {'answer_start': [221], 'text': ['در اواخر دهه 1990']},
 'context': 'بیانسه جیزل نوولز-کارتر (/ biːˈjɒnseɪ / bee-YON-say) (متولد 4 سپتامبر 1981) خواننده ، ترانه سرا ، تهیه کننده ضبط و بازیگر آمریکایی است. وی در هوستون تگزاس متولد و بزرگ شد و در کودکی در مسابقات مختلف آواز و رقص شرکت کرد و در اواخر دهه 1990 به عنوان خواننده اصلی گروه دخترانه R&B Destiny\'s Child به شهرت رسید. با مدیریت پدرش ماتیو نولز ، این گروه به یکی از پرفروش ترین گروه های دختران جهان در تمام دوران تبدیل شد. وقفه آنها باعث انتشار اولین آلبوم بیانسه ، Dangerously in Love (2003) شد ، که او را به عنوان یک هنرمند انفرادی در سراسر جهان تثبیت کرد ، پنج جایزه گرمی را به دست آورد و تک آهنگ های شماره یک بی بیلبورد Hot "Crazy in Love" و "Baby Boy" را به نمایش گذاشت. .',
 'id': '56be85543aeaaa14008c9063',
 'question': 'از چه زمانی بیانسه شروع به محبوبیت کرد؟',
 'title': 'بیانسه'}


**Preprocessing the training data**

In [5]:
model_checkpoint = "HooshvareLab/bert-fa-base-uncased"
max_length = 384 # The maximum length of a feature (question and context) 
doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/440 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

In [7]:
for i, example in enumerate(data["train"]):
    if len(tokenizer(example["question"], example["context"])["input_ids"]) > 384:
        break
example = data["train"][i]

In [8]:
len(tokenizer(example["question"], example["context"])["input_ids"])

439

In [9]:
len(tokenizer(example["question"], example["context"], max_length=max_length, truncation="only_second")["input_ids"])

384

In [10]:
pad_on_right = tokenizer.padding_side == "right"

In [11]:
def prepare_train_features(examples):
    examples["question"] = [q.lstrip() for q in examples["question"]]

    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    offset_mapping = tokenized_examples.pop("offset_mapping")

    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        sequence_ids = tokenized_examples.sequence_ids(i)

        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
           # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])
            
            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1
            
            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [12]:
tokenized_data = data.map(prepare_train_features, batched=True, remove_columns=data["train"].column_names)

  0%|          | 0/65 [00:00<?, ?ba/s]

  0%|          | 0/6 [00:00<?, ?ba/s]

**Fine-tuning the model**

In [13]:
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/624M [00:00<?, ?B/s]

Some weights of the model checkpoint at HooshvareLab/bert-fa-base-uncased were not used when initializing BertForQuestionAnswering: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized 

In [14]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=24,
    per_device_eval_batch_size=24,
    num_train_epochs=2,
    weight_decay=0.01,
)

In [15]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["validation"],
    tokenizer=tokenizer,
)

In [16]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 65491
  Num Epochs = 2
  Instantaneous batch size per device = 24
  Total train batch size (w. parallel, distributed & accumulation) = 24
  Gradient Accumulation steps = 1
  Total optimization steps = 5458


Epoch,Training Loss,Validation Loss
1,1.393600,1.291923
2,1.035700,1.387119


Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-500/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-500/special_tokens_map.json
Saving model checkpoint to ./results/checkpoint-1000
Configuration saved in ./results/checkpoint-1000/config.json
Model weights saved in ./results/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-1000/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-1000/special_tokens_map.json
Saving model checkpoint to ./results/checkpoint-1500
Configuration saved in ./results/checkpoint-1500/config.json
Model weights saved in ./results/checkpoint-1500/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-1500/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-1500/special_toke

TrainOutput(global_step=5458, training_loss=1.3392335158558109, metrics={'train_runtime': 10435.4639, 'train_samples_per_second': 12.552, 'train_steps_per_second': 0.523, 'total_flos': 2.5668878843096064e+16, 'train_loss': 1.3392335158558109, 'epoch': 2.0})

In [17]:
trainer.save_model("model-trained")

Saving model checkpoint to model-trained
Configuration saved in model-trained/config.json
Model weights saved in model-trained/pytorch_model.bin
tokenizer config file saved in model-trained/tokenizer_config.json
Special tokens file saved in model-trained/special_tokens_map.json


**Evaluation**

In [18]:
for batch in trainer.get_eval_dataloader():
    break
batch = {k: v.to(trainer.args.device) for k, v in batch.items()}
with torch.no_grad():
    output = trainer.model(**batch)
output.keys()

odict_keys(['loss', 'start_logits', 'end_logits'])

In [19]:
output.start_logits.shape, output.end_logits.shape

(torch.Size([24, 384]), torch.Size([24, 384]))

In [20]:
output.start_logits.argmax(dim=-1), output.end_logits.argmax(dim=-1)

(tensor([ 48,  35,  72,  15,   0, 141,   0,   0,  83,  18,  30,   0, 101,   0,
          99,  98,  48,  75, 123,  47,  14,  70,  23, 101], device='cuda:0'),
 tensor([ 48,  38,  76,  16,   0, 144, 238, 118,  83,  22,  31,   0, 102, 183,
         100,  99,  48,  77, 127,  48,  14,  70,   0, 103], device='cuda:0'))

In [21]:
n_best_size = 20
max_answer_length = 30

In [22]:
start_logits = output.start_logits[0].cpu().numpy()
end_logits = output.end_logits[0].cpu().numpy()

start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        if start_index <= end_index: 
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": "" 
                }
            )

In [23]:
def prepare_validation_features(examples):

    examples["question"] = [q.lstrip() for q in examples["question"]]

    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):

        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

In [24]:
validation_features = data["validation"].map(
    prepare_validation_features,
    batched=True,
    remove_columns=data["validation"].column_names
)

  0%|          | 0/6 [00:00<?, ?ba/s]

In [25]:
raw_predictions = trainer.predict(validation_features)

The following columns in the test set don't have a corresponding argument in `BertForQuestionAnswering.forward` and have been ignored: offset_mapping, example_id. If offset_mapping, example_id are not expected by `BertForQuestionAnswering.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 5814
  Batch size = 24


In [26]:
validation_features.set_format(type=validation_features.format["type"], columns=list(validation_features.features.keys()))

In [45]:
start_logits = output.start_logits[0].cpu().numpy()
end_logits = output.end_logits[0].cpu().numpy()
offset_mapping = validation_features[0]["offset_mapping"]

context = data["validation"][0]["context"]

start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:

        if (
            start_index >= len(offset_mapping)
            or end_index >= len(offset_mapping)
            or offset_mapping[start_index] is None
            or offset_mapping[end_index] is None
        ):
            continue

        if end_index < start_index or end_index - start_index + 1 > max_answer_length:
            continue
        if start_index <= end_index: 
            start_char = offset_mapping[start_index][0]
            end_char = offset_mapping[end_index][1]
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": context[start_char: end_char]
                }
            )

valid_answers = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[:n_best_size]
valid_answers

[{'score': 10.840162, 'text': 'فرانسه'},
 {'score': 7.730687,
  'text': 'فرانسوی: Normands ؛ لاتین: Normanni) افرادی بودند که در قرون 10 و 11th نام خود را به نرماندی ، منطقه ای در فرانسه'},
 {'score': 4.9917436,
  'text': 'فرانسه گذاشتند. آنها از نژاد مهاجران نورس ("نورمن" از "نورسمن" ناشی می شود) و دزدان دریایی از دانمارک ، ایسلند و نروژ'},
 {'score': 4.670554, 'text': 'فرانسوی'},
 {'score': 4.3119583,
  'text': 'فرانسه گذاشتند. آنها از نژاد مهاجران نورس ("نورمن" از "نورسمن" ناشی می شود) و دزدان دریایی از دانمارک'},
 {'score': 3.9036803,
  'text': 'فرانسه گذاشتند. آنها از نژاد مهاجران نورس ("نورمن" از "نورسمن" ناشی می شود) و دزدان دریایی از دانمارک ، ایسلند'},
 {'score': 2.543005, 'text': 'منطقه ای در فرانسه'},
 {'score': 2.070456, 'text': 'نرماندی ، منطقه ای در فرانسه'},
 {'score': 2.0491552, 'text': 'در فرانسه'},
 {'score': 0.6003432, 'text': 'فرانسه گذاشتند.'},
 {'score': 0.10377073, 'text': 'فرانسه گذاشتند. آنها'},
 {'score': -0.80651253, 'text': 'دانمارک ، ایسلند و نروژ'},
 {'sco

In [46]:
data["validation"][0]["answers"]

{'answer_start': [136, 136, 136, 136],
 'text': ['فرانسه', 'فرانسه', 'فرانسه', 'فرانسه']}

In [47]:
examples = data["validation"]
features = validation_features

example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
features_per_example = collections.defaultdict(list)
for i, feature in enumerate(features):
    features_per_example[example_id_to_index[feature["example_id"]]].append(i)

In [48]:
def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions

    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    predictions = collections.OrderedDict()

    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    for example_index, example in enumerate(tqdm(examples)):

        feature_indices = features_per_example[example_index]

        min_null_score = None
        valid_answers = []
        
        context = example["context"]

        for feature_index in feature_indices:
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]

            offset_mapping = features[feature_index]["offset_mapping"]

            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:

                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue

                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            best_answer = {"text": "", "score": 0.0}
        
        answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
        predictions[example["id"]] = answer

    return predictions

In [49]:
final_predictions = postprocess_qa_predictions(data["validation"], validation_features, raw_predictions.predictions)

Post-processing 5726 example predictions split into 5814 features.


  0%|          | 0/5726 [00:00<?, ?it/s]

In [53]:
formatted_predictions = [{"id": k, "prediction_text": v, "no_answer_probability": 0.0} for k, v in final_predictions.items()]
references = [{"id": ex["id"], "answers": ex["answers"]} for ex in data["validation"]]

#metric.compute(predictions=formatted_predictions, references=references)

In [54]:
from tqdm import tqdm
import torch


class AnswerPredictor:
  def __init__(self, model, tokenizer, device='cuda', n_best=10, max_length=512, stride=256, no_answer=False):
      """Initializes PyTorch Question Answering Prediction
      It's best to leave use the default values.
      Args:
          model: Fine-tuned torch model
          tokenizer: Transformers tokenizer
          device (torch.device): Running device
          n_best (int): Number of best possible answers
          max_length (int): Tokenizer max length
          stride (int): Tokenizer stride
          no_answer (bool): If True, model can return "no answer"
      """
      self.model = model.eval().to(device)
      self.tokenizer = tokenizer
      self.device = device
      self.max_length = max_length
      self.stride = stride
      self.no_answer = no_answer
      self.n_best = n_best


  def model_pred(self, questions, contexts, batch_size=1):
      n = len(contexts)
      if n%batch_size!=0:
          raise Exception("batch_size must be divisible by sample length")

      tokens = self.tokenizer(questions, contexts, add_special_tokens=True, 
                              return_token_type_ids=True, return_tensors="pt", padding=True, 
                              return_offsets_mapping=True, truncation="only_second", 
                              max_length=self.max_length, stride=self.stride)

      start_logits, end_logits = [], []
      for i in tqdm(range(0, n-batch_size+1, batch_size)):
          with torch.no_grad():
              out = self.model(tokens['input_ids'][i:i+batch_size].to(self.device), 
                          tokens['attention_mask'][i:i+batch_size].to(self.device), 
                          tokens['token_type_ids'][i:i+batch_size].to(self.device))

              start_logits.append(out.start_logits)
              end_logits.append(out.end_logits)

      return tokens, torch.stack(start_logits).view(n, -1), torch.stack(end_logits).view(n, -1)


  def __call__(self, questions, contexts, batch_size=1, answer_max_len=100):
      """Creates model prediction
      
      Args: 
          questions (list): Question strings
          contexts (list): Contexts strings
          batch_size (int): Batch size
          answer_max_len (int): Sets the longests possible length for any answer
        
      Returns:
          dict: The best prediction of the model
              (e.g {0: {"text": str, "score": int}})
      """
      tokens, starts, ends = self.model_pred(questions, contexts, batch_size=batch_size)
      start_indexes = starts.argsort(dim=-1, descending=True)[:, :self.n_best]
      end_indexes = ends.argsort(dim=-1, descending=True)[:, :self.n_best]

      preds = {}
      for i, (c, q) in enumerate(zip(contexts, questions)):  
          min_null_score = starts[i][0] + ends[i][0] # 0 is CLS Token
          start_context = tokens['input_ids'][i].tolist().index(self.tokenizer.sep_token_id)
          
          offset = tokens['offset_mapping'][i]
          valid_answers = []
          for start_index in start_indexes[i]:
              if start_index<start_context:
                  continue
              for end_index in end_indexes[i]:
                  if (start_index >= len(offset) or end_index >= len(offset)
                      or offset[start_index] is None or offset[end_index] is None):
                      continue

                  if end_index < start_index or (end_index-start_index+1) > answer_max_len:
                      continue

                  start_char = offset[start_index][0]
                  end_char = offset[end_index][1]
                  valid_answers.append({"score": (starts[i][start_index] + ends[i][end_index]).item(),
                                        "text": c[start_char: end_char]})
                  
          if len(valid_answers) > 0:
              best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
          else:
              best_answer = {"text": "", "score": min_null_score}

          if self.no_answer:
              preds[i] = best_answer if best_answer["score"] >= min_null_score else {"text": "", "score": min_null_score}
          else:
              preds[i] = best_answer

      return preds

In [57]:
text = "سهراب سپهری، در ۱۵ مهر ۱۳۰۷، در کاشان به دنیا آمد.پپدربزرگش میرزا نصرالله‌خان سپهری، نخستین رئیس تلگراف‌خانه کاشان بود. پدرش اسدالله و مادرش ماه‌جبین نام داشتند که هر دو اهل هنر و شعر بودند.دورهٔ ابتدایی را در دبستان خیام کاشان (۱۳۱۹)، و متوسطه را در دبیرستان پهلوی کاشان گذراند و پس از فارغ‌التحصیلی در خرداد ۱۳۲۲ در دورهٔ دوسالهٔ دانش‌سرای مقدماتی پسران، به استخدام ادارهٔ فرهنگ کاشان درآمد."
questions = ["سراب سپهری در چه تاریخی به دنیا آمد؟", "نام پدر سهراب سپهری چیست؟"]

predictor = AnswerPredictor(model, tokenizer, device='cpu', n_best=10)
preds = predictor(questions, [text]*2, batch_size=2)

print()
for k, v in preds.items():
    print(v)

100%|██████████| 1/1 [00:01<00:00,  1.06s/it]


{'score': 11.696466445922852, 'text': '۱۵ مهر ۱۳۰۷'}
{'score': 9.492044448852539, 'text': 'اسدالله'}
